In [89]:
import os
import sys
import copy
import pickle
from queue import SimpleQueue
from typing import List, Tuple, Dict
from sage.graphs.connectivity import connected_components

For saving objects

In [90]:
def pickle_save(obj, filename):
    with open(filename, 'wb') as file:
        pickle.dump(obj, file, protocol=pickle.HIGHEST_PROTOCOL)

In [91]:
def pickle_load(filename):
    with open(filename, 'rb') as file:
        return pickle.load(file)

Util

In [92]:
def generate_transposition(n: int, u: int, v: int) -> Permutation:
    action = list(range(1, n + 1))
    action[u - 1] = v
    action[v - 1] = u
    return Permutation(action)

generate_transposition(4, 2, 4)  # [1, 4, 3, 2]

[1, 4, 3, 2]

In [93]:
indent = 0

def iprint(s: str, pre: int = 0, post: int = 0):
    global indent
    indent += pre
    print("    " * indent + s)
    indent += post

Input verification

In [94]:
def validate_edges(n: int, allowed_pairs: Tuple[Tuple[int, int], ...]):
    """Checks for invalid edges, duplicate edges (up to ordering), and self-loops."""
    elements = [u for pair in allowed_pairs for u in pair]
    if not (1 <= min(elements) and max(elements) <= n):
        print("Element out of range")
        return False

    for (u, v) in allowed_pairs:
        if u == v:
            print(f"Self-loop found: {(u, v)}")
            return False

    pairs = set([(min(u, v), max(u, v)) for (u, v) in allowed_pairs])
    if len(pairs) != len(allowed_pairs):
        iprint("Duplicate edge found.")
        return False

    if not Graph([list(range(1, n + 1)), list(allowed_pairs)]).is_connected():
        print("Graph is not connected")
        return False

    return True

CC = connected component

`dp[rank of permutation, CC's] = (min length, number of factorizations with such length)`

In [95]:
Edge = Tuple[int, int]
State = Tuple[int, Tuple[Tuple[int, ...], ...]]

def merge_cc(cc_list, u: int, v: int):
    cc_u = [cc for cc in cc_list if u in cc]
    cc_v = [cc for cc in cc_list if v in cc]
    assert len(cc_u) == 1 and len(cc_v) == 1
    cc_u = cc_u[0]
    cc_v = cc_v[0]
    
    if cc_u == cc_v:
        return cc_list
    new_cc_list = list(cc_list)
    new_cc_list.remove(cc_u)
    new_cc_list.remove(cc_v)
    new_cc = tuple(sorted(cc_u + cc_v))
    new_cc_list.append(new_cc)
    new_cc_list.sort()
    return tuple(map(tuple, new_cc_list))


def get_next_states(n, allowed_edges, state):
    next_states = []
    rank, cc_list = state
    perm = Permutations(n).unrank(rank)
    for (u, v) in allowed_edges:
        action = generate_transposition(n, u, v)
        new_perm = Permutation(perm).left_action_product(action)
        new_cc = merge_cc(cc_list, u, v)
        next_states.append((new_perm.rank(), new_cc))
    return next_states


def compute_dp(n: int, allowed_edges: List[Edge]) -> Dict[State, Tuple[int, int]]:
    completed_perms = set()
    base_case = (0, tuple((i,) for i in range(1, n + 1)))
    dp = {base_case: (0, 1)}
    queue = SimpleQueue()
    queue.put(base_case)
    
    while True:
        assert not queue.empty()
        state = queue.get()
        rank, cc_list = state
        perm = Permutations(n).unrank(rank)
        min_length, count = dp[state]

        next_states = get_next_states(n, allowed_edges, state)
        for ns in next_states:
            if ns not in dp:
                dp[ns] = (min_length + 1, count)
                queue.put(ns)
            else:
                next_min_length = dp[ns][0]
                assert next_min_length <= min_length + 1
                if next_min_length == min_length + 1:
                    dp[ns] = (dp[ns][0], dp[ns][1] + count)
        
        if len(cc_list) == 1:
            completed_perms.add(perm)

        # Check that we can terminate
        if len(completed_perms) == factorial(n):
            break
    
    return dp


`compute_length_n(n)`: Returns a map from permutation rank (int) to (min length of factorization, number of such factorizations).

In [105]:
def compute_length_n(n: int, allowed_edges: List[Tuple[int, int]]):
    global indent
    indent = 0
    target_cc_list = (tuple(range(1, n + 1)),)

    dp = compute_dp(n, allowed_edges)
    result = {rank: dp[(rank, target_cc_list)] for rank in range(factorial(n))}  # rank to (min length, # of facts)
    return result

In [106]:
def compute_length_n_graph(n: int, graph: Graph):
    edges = list(graph.edges(labels=False))
    for (i, (u, v)) in enumerate(edges):
        edges[i] = (u + 1, v + 1)
    return compute_length_n(n, edges)

For verifying star transpositions

In [107]:
from functools import reduce
from operator import mul

def star_count(perm: List[int]):
    """Return the number of minimal transitive star factorizations of perm."""
    perm = Permutation(perm)
    cycles = perm.to_cycles()
    cycle_lengths = [len(cycle) for cycle in cycles]
    n = len(perm)
    m = len(cycles)
    return factorial(n + m - 2) / factorial(n) * reduce(mul, cycle_lengths)

Try all graphs of size $n$.

In [115]:
min_n = 4
max_n = 6

graph_db = GraphDatabase()
result = {}
for n in range(min_n, max_n + 1):
    print(f'Starting n = {n}')
    graph_query = GraphQuery(graph_db=graph_db, display_cols=[], num_vertices=['=', n], vertex_connectivity=1)
    graph_list = graph_query.get_graphs_list()
    for g in graph_list:
        result[g.copy(immutable=True)] = compute_length_n_graph(n, g)
    pickle_save(result, f'all_graphs_n_{min_n}_to_{max_n}.pkl')

Starting n = 4
Starting n = 5
Starting n = 6


In [118]:
# Load file
result = pickle_load(f'all_graphs_n_4_to_6.pkl')
g = Graph([(0, 1), (0, 2), (0, 3)])
print(result[g.copy(immutable=True)])

{Graph on 4 vertices: {0: (6, 30), 1: (5, 10), 2: (5, 10), 3: (4, 3), 4: (4, 3), 5: (5, 10), 6: (5, 10), 7: (4, 4), 8: (4, 3), 9: (3, 1), 10: (3, 1), 11: (4, 3), 12: (4, 3), 13: (3, 1), 14: (5, 10), 15: (4, 3), 16: (4, 4), 17: (3, 1), 18: (3, 1), 19: (4, 3), 20: (4, 3), 21: (5, 10), 22: (3, 1), 23: (4, 4)}, Graph on 4 vertices: {0: (6, 42), 1: (5, 14), 2: (5, 14), 3: (4, 4), 4: (4, 4), 5: (5, 11), 6: (5, 6), 7: (6, 16), 8: (4, 3), 9: (3, 1), 10: (5, 5), 11: (4, 3), 12: (4, 3), 13: (5, 5), 14: (5, 11), 15: (4, 4), 16: (4, 2), 17: (3, 2), 18: (3, 1), 19: (4, 3), 20: (4, 4), 21: (5, 14), 22: (3, 2), 23: (4, 6)}, Graph on 4 vertices: {0: (6, 210), 1: (5, 45), 2: (5, 50), 3: (4, 11), 4: (4, 11), 5: (5, 45), 6: (5, 45), 7: (4, 8), 8: (4, 11), 9: (3, 4), 10: (3, 1), 11: (4, 13), 12: (4, 11), 13: (3, 1), 14: (5, 45), 15: (4, 13), 16: (4, 8), 17: (3, 4), 18: (3, 4), 19: (4, 13), 20: (4, 13), 21: (5, 70), 22: (3, 4), 23: (4, 20)}, Graph on 5 vertices: {0: (8, 336), 1: (7, 84), 2: (7, 84), 3: (6,

KeyError: Graph on 4 vertices

In [ ]:
def main(n: int, allowed_edges: Tuple[Tuple[int, int], ...]):
    if not validate_edges(n, allowed_edges):
        return
    results = compute_length_n(n, allowed_edges)[0]

    div_cnt = 0
    for perm, (min_length, count) in results.items():
        sc = star_count(perm)
        print(f"{perm}: two-root = {count}, star = {sc}, divisible={count % sc == 0}, q={count / sc}")
        if count % sc == 0:
            div_cnt += 1
        # For star transpositions:
        # expected = star_count(perm)
        # if count != expected:
        #     print(f">>>>>> WARNING: expected {expected}, actual = {count}")

    print(f'div_cnt={div_cnt}, probability={div_cnt / factorial(n)}')
        